# Price Elasticity Analysis - Notebook 1: Data Loading

This notebook focuses on loading and initial inspection of data for price elasticity analysis at Thai Union Group.

## Objectives
- Load raw sales data from various sources
- Perform initial data quality assessment
- Document data structure and characteristics
- Prepare data for subsequent cleaning and analysis

## Business Context
Price elasticity analysis will help Thai Union Group understand how demand responds to price changes across different product categories, enabling better pricing strategies and revenue optimization.


In [1]:
# Data refreshed: 2025-06-24
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from pathlib import Path

# Add project root to Python path so we can import src modules
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.utils.data_helpers import load_config, load_sales_data

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set matplotlib backend for inline plotting
%matplotlib inline

# Set seaborn style
sns.set_style("whitegrid")

print("Libraries imported successfully!")
print(f"Working directory: {os.getcwd()}")
print(f"Python version: {sys.version}")

# Check if required libraries are available
try:
    import pyodbc
    print("✓ pyodbc available for Access database connection")
except ImportError:
    print("⚠ pyodbc not installed. Run: pip install pyodbc")

try:
    import pyarrow
    print("✓ pyarrow available for Parquet format")
except ImportError:
    print("⚠ pyarrow not installed. Run: pip install pyarrow")


Libraries imported successfully!
Working directory: C:\Users\adaves\OneDrive - Thai Union Group\Documents\elasticity\notebooks
Python version: 3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]
✓ pyodbc available for Access database connection
✓ pyarrow available for Parquet format


In [2]:
# Database connection and data extraction
import pyodbc
import time
from pathlib import Path

# Database configuration
DB_PATH = r"C:\Users\adaves\Thai Union Group\COSI - Sales Planning Team - General\Sales Toolbox 2020 - IRI.accdb"
TABLE_NAME = "tblIRI2"

# Output paths
RAW_DATA_DIR = Path("../data/raw")
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_FILE = RAW_DATA_DIR / "iri_sales_data.parquet"

print(f"Database path: {DB_PATH}")
print(f"Table: {TABLE_NAME}")
print(f"Output file: {OUTPUT_FILE}")
print(f"Expected rows: ~1,960,393")


Database path: C:\Users\adaves\Thai Union Group\COSI - Sales Planning Team - General\Sales Toolbox 2020 - IRI.accdb
Table: tblIRI2
Output file: ..\data\raw\iri_sales_data.parquet
Expected rows: ~1,960,393


In [3]:
def extract_from_access():
    """Extract data from Access database and save as Parquet."""
    
    # Check if file already exists
    if OUTPUT_FILE.exists():
        print(f"✓ Data file already exists: {OUTPUT_FILE}")
        file_size = OUTPUT_FILE.stat().st_size / (1024**2)  # MB
        print(f"  File size: {file_size:.1f} MB")
        
        # Load and check row count
        df_check = pd.read_parquet(OUTPUT_FILE)
        print(f"  Rows in file: {len(df_check):,}")
        return df_check
    
    print("🔄 Extracting data from Access database...")
    start_time = time.time()
    
    try:
        # Connect to Access database
        conn_string = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={DB_PATH};'
        conn = pyodbc.connect(conn_string)
        
        print("✓ Connected to database")
        
        # Extract data using pandas
        query = f"SELECT * FROM {TABLE_NAME}"
        df = pd.read_sql_query(query, conn)
        
        conn.close()
        
        # Save as Parquet
        df.to_parquet(OUTPUT_FILE, index=False)
        
        # Performance metrics
        extraction_time = time.time() - start_time
        file_size = OUTPUT_FILE.stat().st_size / (1024**2)  # MB
        
        print(f"✅ Data extraction complete!")
        print(f"   Rows extracted: {len(df):,}")
        print(f"   Columns: {len(df.columns)}")
        print(f"   File size: {file_size:.1f} MB")
        print(f"   Extraction time: {extraction_time:.1f} seconds")
        
        return df
        
    except Exception as e:
        print(f"❌ Error extracting data: {e}")
        return None

# Execute extraction
df_raw = extract_from_access()


✓ Data file already exists: ..\data\raw\iri_sales_data.parquet
  File size: 101.7 MB


  Rows in file: 1,979,310


In [4]:
# Initial data inspection
if df_raw is not None:
    print("📊 DATA OVERVIEW")
    print(f"Shape: {df_raw.shape}")
    print(f"Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    print("\n📋 COLUMN INFO:")
    print(df_raw.dtypes)
    print("\n🔍 FIRST FEW ROWS:")
    display(df_raw.head())
    
    print("\n📈 SAMPLE STATISTICS:")
    display(df_raw.describe(include='all'))


📊 DATA OVERVIEW
Shape: (1979310, 36)


Memory usage: 1182.0 MB

📋 COLUMN INFO:
ID                                                   int64
Geography                                           object
Product                                             object
Time                                                object
Geography Key                                      float64
Product Key                                         object
Unit Sales                                         float64
Unit Sales Year Ago                                float64
Unit Sales 2 Years Ago                             float64
Unit Sales 3 Years Ago                             float64
Base Unit Sales                                    float64
Incremental Units                                  float64
Volume Sales                                       float64
% Stores                                           float64
ACV Weighted Distribution Feature Only             float64
ACV Weighted Distribution Feature and Display      float64
ACV Weighted Dis

,ID,Geography,Product,Time,Geography Key,Product Key,Unit Sales,Unit Sales Year Ago,Unit Sales 2 Years Ago,Unit Sales 3 Years Ago,Base Unit Sales,Incremental Units,Volume Sales,% Stores,ACV Weighted Distribution Feature Only,ACV Weighted Distribution Feature and Display,ACV Weighted Distribution Display Only,ACV Weighted Distribution Price Reductions Only,ACV Weighted Distribution,Price per Unit,Price per Unit Year Ago,Price per Unit Any Merch,Price per Unit No Merch,Total Points of Distribution,Total Points of Distribution Change vs YA,Dollar Sales,Dollar Sales Year Ago,Dollar Sales 2 Years Ago,Dollar Sales 3 Years Ago,Incremental Dollars,Base Dollar Sales,Unit Sales per Pt of Distribution,Weighted Average Base Price Per Unit,Opportunity Dollars,Dollar Trade Efficiency,Unit Trade Efficiency
0,10147576,Total US - Multi Outlet,ACE OF DIAMONDS CHUNK LIGHT WATER 5 OZ 1 CT CA...,Week Ending 12-31-23,609857.0,452471233:225241880:59659909:460977384:7080095...,64.0,13.0,165.0,1132.0,64.0,NaN,318.0,0.04,NaN,NaN,NaN,NaN,0.02,1.67,1.52,NaN,1.67,0.0,0.0,106.0,19.0,186.0,937.0,NaN,106.0,970.0,1.67,270117.0,NaN,NaN
1,10147577,Total US - Multi Outlet,ACE OF DIAMONDS CHUNK LIGHT WATER 5 OZ 1 CT CA...,Week Ending 01-07-24,609857.0,452471233:225241880:59659909:460977384:7080095...,20.0,21.0,185.0,1816.0,20.0,NaN,99.0,0.02,NaN,NaN,NaN,NaN,0.01,1.62,1.49,NaN,1.62,0.0,0.0,32.0,32.0,170.0,1534.0,NaN,32.0,527.0,1.62,152402.0,NaN,NaN
2,10147578,Total US - Multi Outlet,CHICKEN OF THE SEA CHUNK LIGHT OIL 12 OZ 1 CT ...,Week Ending 12-31-23,609857.0,452471233:225241880:59659909:460977384:7080095...,8431.0,12740.0,16067.0,22362.0,8415.0,16.0,101175.0,2.74,NaN,NaN,NaN,0.13,12.15,2.97,2.96,3.49,2.96,12.2,-0.5,25022.0,37736.0,39876.0,57626.0,53.0,24969.0,424.0,2.97,237774.0,0.21,0.19
3,10147579,Total US - Multi Outlet,CHICKEN OF THE SEA CHUNK LIGHT OIL 12 OZ 1 CT ...,Week Ending 01-07-24,609857.0,452471233:225241880:59659909:460977384:7080095...,13247.0,16638.0,19656.0,29992.0,13159.0,88.0,158961.0,3.45,NaN,NaN,NaN,0.35,16.04,2.99,2.93,3.19,2.98,16.0,0.6,39594.0,48689.0,48848.0,74517.0,235.0,39360.0,547.0,2.99,320249.0,0.59,0.66
4,10147580,Total US - Multi Outlet,CHICKEN OF THE SEA CHUNK LIGHT OIL 5 OZ 1 CT C...,Week Ending 12-31-23,609857.0,452471233:225241880:59659909:460977384:7080095...,70809.0,138071.0,175360.0,179849.0,68642.0,2168.0,354047.0,12.21,0.05,NaN,0.03,1.42,34.64,1.32,1.27,1.13,1.34,34.6,-21.9,93769.0,175791.0,183822.0,177596.0,2081.0,91687.0,990.0,1.34,182682.0,2.22,3.06



📈 SAMPLE STATISTICS:


,ID,Geography,Product,Time,Geography Key,Product Key,Unit Sales,Unit Sales Year Ago,Unit Sales 2 Years Ago,Unit Sales 3 Years Ago,Base Unit Sales,Incremental Units,Volume Sales,% Stores,ACV Weighted Distribution Feature Only,ACV Weighted Distribution Feature and Display,ACV Weighted Distribution Display Only,ACV Weighted Distribution Price Reductions Only,ACV Weighted Distribution,Price per Unit,Price per Unit Year Ago,Price per Unit Any Merch,Price per Unit No Merch,Total Points of Distribution,Total Points of Distribution Change vs YA,Dollar Sales,Dollar Sales Year Ago,Dollar Sales 2 Years Ago,Dollar Sales 3 Years Ago,Incremental Dollars,Base Dollar Sales,Unit Sales per Pt of Distribution,Weighted Average Base Price Per Unit,Opportunity Dollars,Dollar Trade Efficiency,Unit Trade Efficiency
count,1.979310e+06,1979310,1979310,1979310,1.979310e+06,1979310,1.924807e+06,1.760022e+06,1.645534e+06,1.561477e+06,1.918517e+06,1.061070e+06,1.924807e+06,1.924807e+06,86360.000000,25536.000000,246684.000000,981197.000000,1.924806e+06,1.924807e+06,1.760022e+06,1.061070e+06,1.875642e+06,1.924806e+06,1.958121e+06,1.924807e+06,1.760022e+06,1.645534e+06,1.561477e+06,1.061070e+06,1.918517e+06,1.924806e+06,1.924807e+06,1.924806e+06,1.061070e+06,1.061070e+06
unique,NaN,86,3591,181,NaN,3543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Total US - Multi Outlet+,STARKIST TUNA CREATIONS CHUNK LIGHT MARINATED ...,Week Ending 02-25-24,NaN,452471233:225241880:24106645:460977384:7080095...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,447009,5201,30703,NaN,5201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.116120e+07,NaN,NaN,NaN,1.898079e+07,NaN,4.235440e+03,4.645063e+03,5.197768e+03,5.506524e+03,3.489925e+03,1.370517e+03,2.753375e+04,2.139496e+01,14.938572,4.646545,2.376926,8.594143,2.389187e+01,3.988806e+00,3.882649e+00,3.223260e+00,4.058921e+00,2.389066e+01,1.113718e+00,9.002883e+03,9.737486e+03,1.046946e+04,1.048909e+04,1.681034e+03,8.100119e+03,4.773763e+02,4.100139e+00,2.008702e+05,9.196698e+00,1.229597e+01
std,5.958641e+05,NaN,NaN,NaN,1.330206e+07,NaN,3.141340e+04,3.381335e+04,3.881551e+04,4.118375e+04,2.284303e+04,1.809792e+04,2.038063e+05,3.069497e+01,29.262681,12.982583,7.907149,19.844399,3.184263e+01,3.525178e+00,3.385575e+00,2.955124e+00,3.543367e+00,3.184351e+01,1.306069e+01,5.038263e+04,5.243977e+04,5.672727e+04,5.669755e+04,2.328828e+04,4.158533e+04,3.599418e+03,3.588279e+00,1.325008e+06,1.560965e+01,1.785737e+01
min,1.014758e+07,NaN,NaN,NaN,1.478300e+04,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000e-01,1.000000e-01,5.000000e-02,5.000000e-02,0.000000e+00,-1.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.861500e+04,0.000000e+00,0.000000e+00,1.000000e-01,0.000000e+00,-7.750400e+02,0.000000e+00
25%,1.064243e+07,NaN,NaN,NaN,6.098590e+05,NaN,3.700000e+01,5.100000e+01,5.500000e+01,6.100000e+01,3.500000e+01,2.000000e+00,2.040000e+02,2.100000e-01,0.070000,0.030000,0.020000,0.080000,3.600000e-01,1.990000e+00,1.990000e+00,1.500000e+00,2.000000e+00,4.000000e-01,-6.000000e-01,1.350000e+02,1.820000e+02,1.860000e+02,1.910000e+02,3.000000e+00,1.290000e+02,6.000000e+00,2.000000e+00,7.710000e+02,1.500000e-01,3.900000e-01
50%,1.114832e+07,NaN,NaN,NaN,2.747779e+07,NaN,2.240000e+02,2.710000e+02,3.010000e+02,3.300000e+02,2.050000e+02,1.700000e+01,1.220000e+03,2.410000e+00,0.670000,0.240000,0.090000,0.820000,4.220000e+00,2.990000e+00,2.980000e+00,2.490000e+00,3.010000e+00,4.200000e+00,0.000000e+00,7.150000e+02,8.370000e+02,8.880000e+02,8.990000e+02,3.500000e+01,6.650000e+02,3.800000e+01,3.040000e+00,1.142100e+04,2.050000e+00,3.210000e+00
75%,1.167793e+07,NaN,NaN,NaN,2.811395e+07,NaN,

In [5]:
# !pip install pyarrow

In [6]:
# Install pyodbc using pip
# !pip install pyodbc